In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import lit

In [0]:
%fs
ls /Volumes/workspace/ecommerce/ecommerce_data

path,name,size,modificationTime
dbfs:/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv,2019-Nov.csv,9006762395,1767962633000
dbfs:/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv,2019-Oct.csv,5668612855,1767962553000
dbfs:/Volumes/workspace/ecommerce/ecommerce_data/delta/,delta/,0,1768266063342
dbfs:/Volumes/workspace/ecommerce/ecommerce_data/parquet/,parquet/,0,1768266063342


In [0]:
df_oct = spark.read.parquet("/Volumes/workspace/ecommerce/ecommerce_data/parquet/oct/")
df_oct.limit(5).show()

+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|2019-10-13 06:25:46|      view|   1002544|2053013555631882655|electronics.smart...|   apple| 460.51|518958788|e7e27c5c-1e78-481...|
|2019-10-13 06:25:46|      view|   3700301|2053013565983425517|appliances.enviro...|   vitek| 120.93|557977070|7afc206c-7259-4be...|
|2019-10-13 06:25:46|      view|  49100004|2127425375913902544|                NULL|    NULL|  45.05|514456508|9d6837a5-40df-49d...|
|2019-10-13 06:25:46|      view|   9200409|2053013552913973497|computers.periphe...|defender|  12.56|512530774|df2d048d-c1ae-41b...|
|2019-10-13 06:25:46|      view|   1306558|2053013558920217191|  comp

#### There is NO event_id column in your table. Hence, The Schema is Clean

In [0]:
from delta.tables import DeltaTable

deltaTable = DeltaTable.forName(spark, "workspace.ecommerce.events_oct")

deltaTable.alias("t").merge(
    source = spark.table("updates_view").alias("s"),
    condition = "t.event_id = s.event_id"
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4825006384483855>, line 10
      1 from delta.tables import DeltaTable
      3 deltaTable = DeltaTable.forName(spark, "workspace.ecommerce.events_oct")
      5 deltaTable.alias("t").merge(
      6     source = spark.table("updates_view").alias("s"),
      7     condition = "t.event_id = s.event_id"
      8 ).whenMatchedUpdateAll() \
      9  .whenNotMatchedInsertAll() \
---> 10  .execute()

File /databricks/python/lib/python3.12/site-packages/delta/connect/tables.py:609, in DeltaMergeBuilder.execute(self)
    599 plan = MergeIntoTable(
    600     self._target,
    601     self._source,
   (...)
    606     self._with_schema_evolution
    607 )
    608 df = DataFrame(plan, session=self._spark)
--> 609 return self._spark.createDataFrame(df.toPandas())

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect

In [0]:
updates = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/incremental/new_data.csv",header=True,inferSchema=True)

In [0]:
from pyspark.sql.functions import to_timestamp, col

updates_fixed = updates.withColumn(
    "event_time",
    to_timestamp(col("event_time"), "dd-MM-yyyy HH:mm")
)

In [0]:
updates_fixed.filter(col("event_time").isNull()).show(truncate=False)

+----------+----------+----------+-----------+-------------+-----+-----+-------+------------+
|event_time|event_type|product_id|category_id|category_code|brand|price|user_id|user_session|
+----------+----------+----------+-----------+-------------+-----+-----+-------+------------+
+----------+----------+----------+-----------+-------------+-----+-----+-------+------------+



## Ingest → DataFrame → Temp View → MERGE
### We use createOrReplaceTempView to turn a DataFrame into a SQL-accessible object so it can be used in MERGE and other SQL operations.


In [0]:
from pyspark.sql.functions import lit
from delta.tables import DeltaTable

updates_fixed = updates_fixed.withColumn("source_system", lit("incremental_csv"))

updates_fixed.createOrReplaceTempView("updates_view")

deltaTable = DeltaTable.forName(spark, "workspace.ecommerce.events_oct")

deltaTable.alias("t").merge(
    source = spark.table("updates_view").alias("s"),
    condition = """
        t.user_session = s.user_session
        AND t.event_time = s.event_time
        AND t.product_id = s.product_id
    """
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
result_df = spark.table("workspace.ecommerce.events_oct") \
    .filter(col("user_session") == "abc-session-1")

display(result_df.orderBy("event_time"))


event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,source_system
2019-10-20T00:00:00.000Z,view,1002589,2053010000000000000,electronics.smartphone,samsung,460.5,51424849,abc-session-1,incremental_csv


In [0]:
display(spark.table("workspace.ecommerce.events_oct").limit(10))

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,source_system
2019-10-29T09:17:32.000Z,view,1004858,2053013555631882655,electronics.smartphone,samsung,131.02,559389453,d71e7146-3d1a-4b96-9d87-fdd314d4694c,null
2019-10-29T09:17:32.000Z,view,1004858,2053013555631882655,electronics.smartphone,samsung,131.02,565343162,cfbbc662-af26-4ee7-be22-7a3d507bb2bf,null
2019-10-29T09:17:33.000Z,purchase,5800792,2053013553945772349,electronics.audio.subwoofer,null,162.17,536723169,567d676d-5c64-4646-adb2-c8c6ff9fc8fb,null
2019-10-29T09:17:33.000Z,view,1005064,2053013555631882655,electronics.smartphone,xiaomi,187.65,513355340,74a71cdb-c262-4f96-8e58-00bfeacf05e3,null
2019-10-29T09:17:33.000Z,view,1003315,2053013555631882655,electronics.smartphone,apple,965.17,512786243,3331167a-db7f-4b4a-bc31-30f4c4aa93f3,null
2019-10-29T09:17:33.000Z,view,51800024,2135658542386905834,null,tomfarr,121.75,551703117,23ff3e46-7999-4b37-aa3e-8091fc39d1e2,null
2019-10-29T09:17:33.000Z,view,4700478,2053013560899928785,auto.accessories.videoregister,sho-me,141.91,512475445,54924b97-2003-46dd-8261-86ef699d9db6,null
2019-10-29T09:17:33.000Z,view,1004249,2053013555631882655,electronics.smartphone,apple,725.12,550686150,730e7bc6-3df6-4097-a29f-af3283b3891c,null
2019-10-29T09:17:33.000Z,view,1003747,2053013555631882655,electronics.smartphone,sony,282.81,560569926,89ec64f0-f283-4178-8f18-30442a9d128e,null
2019-10-29T09:17:33.000Z,view,1307341,2053013558920217191,computers.notebook,asus,617.75,535175444,71c0c0ac-d7e1-4294-a977-1db78caa97b9,null


In [0]:
result_df.groupBy('source_system').count().show()

+---------------+-----+
|  source_system|count|
+---------------+-----+
|incremental_csv|    1|
+---------------+-----+

